In [1]:
%run Common.ipynb

/data/home/svanbodegraven/notebooks/HealthyHabitatAI/.env
healthyhabitatanimals
healthyhabitatparragrass
ghNvkPl4DYLUUJh55y4pSg3BVV8TtPcHcX96P/zaDPXr9uuREdkGSCaao59d6xfZOfg4l9um1qfBXuNM9VGTTg==
jpEJGrFNi6PNLGOOD62rFBxjHGKTR4+DcppQNQQXi3P/GFmINKfqf8EZJoo73doIAnntv03ZSUsAC442IRR87Q==
processed
processed
f8fc2cd4db6b45a59d1ddd69a8685ddd
4610573e-864f-4f53-9f6a-c61ce520ed5a
5aeb092f-e91d-400f-bd88-e9516b108f20


In [2]:
import numpy as np
import matplotlib.pyplot as plt

from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry
from PIL import Image, ImageFilter

In [3]:
%matplotlib inline

### Blob Storage

#### Para Grass Containers - *Site-Season*

In [4]:
para_grass_containers = blob_service_list_containers(healthy_habitat_paragrass_storage_name, healthy_habitat_paragrass_storage_key)

for container_name in para_grass_containers:
    print(container_name)

cannon-hill-bangkerreng
cannon-hill-kunumeleng
cannon-hill-wurrkeng
jabiru-dreaming-bangkerreng
jabiru-dreaming-kunumeleng
jabiru-dreaming-wurrkeng
resized
ubir-bangkerreng
ubir-kunumeleng
ubir-wurrkeng


### Resize

In [5]:
for container_name in para_grass_containers:
    if container_name != 'resized':
        blobs = blob_service_list_blobs(healthy_habitat_paragrass_storage_name, healthy_habitat_paragrass_storage_key, container_name)

        folders = set([blob.split('/')[0] for blob in blobs])

        for folder in folders:
            interim_folder_path = os.path.join(Path(os.getcwd()).parent, 'data', 'interim', container_name, folder)
            print(interim_folder_path)
            if not os.path.exists(interim_folder_path):
                os.makedirs(interim_folder_path)

        for blob_name in blobs:
            folder = blob_name.split('/')[0]
            file = blob_name.split('/')[1]

            # https://docs.microsoft.com/en-us/rest/api/storageservices/understanding-the-table-service-data-model#characters-disallowed-in-key-fields
            row_key = '{0}-{1}'.format(folder, file)

            filter = "PartitionKey eq '{0}' and RowKey eq '{1}'".format(container_name, row_key)

            entities = table_service_query_entities(healthy_habitat_paragrass_storage_name,
                                                    healthy_habitat_paragrass_storage_key,
                                                    healthy_habitat_paragrass_processed_table_name,
                                                    filter)

            count = len([entity for entity in entities])

            if count == 0:
                print('Processing {0}'.format(blob_name))
                file_path = os.path.join(interim_folder_path, file)

                if not os.path.exists(file_path):
                    blob_service_get_blob_to_path(healthy_habitat_paragrass_storage_name,
                                                  healthy_habitat_paragrass_storage_key,
                                                  container_name,
                                                  blob_name,
                                                  file_path)

                edge = 1024

                image = Image.open(file_path)
                image = image.crop((0, 0, image.size[1], image.size[1])) #left, upper, right, lower
                image = image.resize((edge, edge), resample=Image.LANCZOS)

                buffer = io.BytesIO()
                image.save(buffer, format='JPEG')

                path = '{0}/{1}/{2}'.format(container_name, folder, file)

                blob_service_create_blob_from_bytes(healthy_habitat_paragrass_storage_name,
                                                    healthy_habitat_paragrass_storage_key,
                                                    'resized',
                                                    path,
                                                    buffer.getvalue())

                entity = {'PartitionKey': container_name, 'RowKey': row_key, 'SemanticSegmentation': True}

                table_service_insert(healthy_habitat_paragrass_storage_name,
                                     healthy_habitat_paragrass_storage_key,
                                     healthy_habitat_paragrass_processed_table_name,
                                     entity)

                os.remove(file_path)

/data/home/svanbodegraven/notebooks/HealthyHabitatAI/data/interim/cannon-hill-bangkerreng/2019-04-03-1050
Processing 2019-04-03-1050/DJI_0198.JPG
Processing 2019-04-03-1050/DJI_0199.JPG
Processing 2019-04-03-1050/DJI_0200.JPG
Processing 2019-04-03-1050/DJI_0201.JPG
Processing 2019-04-03-1050/DJI_0202.JPG
Processing 2019-04-03-1050/DJI_0203.JPG
Processing 2019-04-03-1050/DJI_0204.JPG
Processing 2019-04-03-1050/DJI_0205.JPG
Processing 2019-04-03-1050/DJI_0206.JPG
Processing 2019-04-03-1050/DJI_0207.JPG
Processing 2019-04-03-1050/DJI_0208.JPG
Processing 2019-04-03-1050/DJI_0209.JPG
Processing 2019-04-03-1050/DJI_0210.JPG
Processing 2019-04-03-1050/DJI_0211.JPG
Processing 2019-04-03-1050/DJI_0212.JPG
Processing 2019-04-03-1050/DJI_0213.JPG
Processing 2019-04-03-1050/DJI_0214.JPG
Processing 2019-04-03-1050/DJI_0215.JPG
Processing 2019-04-03-1050/DJI_0216.JPG
Processing 2019-04-03-1050/DJI_0217.JPG
Processing 2019-04-03-1050/DJI_0218.JPG
Processing 2019-04-03-1050/DJI_0219.JPG
Processing 201

Processing 2019-04-03-1050/DJI_0401.JPG
Processing 2019-04-03-1050/DJI_0402.JPG
Processing 2019-04-03-1050/DJI_0403.JPG
Processing 2019-04-03-1050/DJI_0404.JPG
Processing 2019-04-03-1050/DJI_0405.JPG
Processing 2019-04-03-1050/DJI_0406.JPG
Processing 2019-04-03-1050/DJI_0407.JPG
Processing 2019-04-03-1050/DJI_0408.JPG
Processing 2019-04-03-1050/DJI_0409.JPG
Processing 2019-04-03-1050/DJI_0410.JPG
Processing 2019-04-03-1050/DJI_0411.JPG
Processing 2019-04-03-1050/DJI_0412.JPG
Processing 2019-04-03-1050/DJI_0413.JPG
Processing 2019-04-03-1050/DJI_0414.JPG
Processing 2019-04-03-1050/DJI_0415.JPG
Processing 2019-04-03-1050/DJI_0416.JPG
Processing 2019-04-03-1050/DJI_0417.JPG
Processing 2019-04-03-1050/DJI_0418.JPG
Processing 2019-04-03-1050/DJI_0419.JPG
Processing 2019-04-03-1050/DJI_0420.JPG
Processing 2019-04-03-1050/DJI_0421.JPG
Processing 2019-04-03-1050/DJI_0422.JPG
Processing 2019-04-03-1050/DJI_0423.JPG
Processing 2019-04-03-1050/DJI_0424.JPG
Processing 2019-04-03-1050/DJI_0425.JPG


Processing 2018-12-05-1540/DJI_0121.JPG
Processing 2018-12-05-1540/DJI_0122.JPG
Processing 2018-12-05-1540/DJI_0123.JPG
Processing 2018-12-05-1540/DJI_0124.JPG
Processing 2018-12-05-1540/DJI_0125.JPG
Processing 2018-12-05-1540/DJI_0126.JPG
Processing 2018-12-05-1540/DJI_0127.JPG
Processing 2018-12-05-1540/DJI_0128.JPG
Processing 2018-12-05-1540/DJI_0129.JPG
Processing 2018-12-05-1540/DJI_0130.JPG
Processing 2018-12-05-1540/DJI_0131.JPG
Processing 2018-12-05-1540/DJI_0132.JPG
Processing 2018-12-05-1540/DJI_0133.JPG
Processing 2018-12-05-1540/DJI_0134.JPG
Processing 2018-12-05-1540/DJI_0135.JPG
Processing 2018-12-05-1540/DJI_0136.JPG
Processing 2018-12-05-1540/DJI_0137.JPG
Processing 2018-12-05-1540/DJI_0138.JPG
Processing 2018-12-05-1540/DJI_0139.JPG
Processing 2018-12-05-1540/DJI_0140.JPG
Processing 2018-12-05-1540/DJI_0141.JPG
Processing 2018-12-05-1540/DJI_0142.JPG
Processing 2018-12-05-1540/DJI_0143.JPG
Processing 2018-12-05-1540/DJI_0144.JPG
Processing 2018-12-05-1540/DJI_0145.JPG


Processing 2018-12-05-1540/DJI_0326.JPG
Processing 2018-12-05-1540/DJI_0327.JPG
Processing 2018-12-05-1540/DJI_0328.JPG
Processing 2018-12-05-1540/DJI_0329.JPG
Processing 2018-12-05-1540/DJI_0330.JPG
Processing 2018-12-05-1540/DJI_0331.JPG
Processing 2018-12-05-1540/DJI_0332.JPG
Processing 2018-12-05-1540/DJI_0333.JPG
Processing 2018-12-05-1540/DJI_0334.JPG
Processing 2018-12-05-1540/DJI_0335.JPG
Processing 2018-12-05-1540/DJI_0336.JPG
Processing 2018-12-05-1540/DJI_0337.JPG
Processing 2018-12-05-1540/DJI_0338.JPG
Processing 2018-12-05-1540/DJI_0339.JPG
Processing 2018-12-05-1540/DJI_0340.JPG
Processing 2018-12-05-1540/DJI_0341.JPG
Processing 2018-12-05-1540/DJI_0342.JPG
Processing 2018-12-05-1540/DJI_0343.JPG
Processing 2018-12-05-1540/DJI_0344.JPG
Processing 2018-12-05-1540/DJI_0345.JPG
Processing 2018-12-05-1540/DJI_0346.JPG
Processing 2018-12-05-1540/DJI_0347.JPG
Processing 2018-12-05-1540/DJI_0348.JPG
Processing 2018-12-05-1540/DJI_0349.JPG
Processing 2018-12-05-1540/DJI_0350.JPG


Processing 2019-08-13-1200/DJI_0081.JPG
Processing 2019-08-13-1200/DJI_0082.JPG
Processing 2019-08-13-1200/DJI_0083.JPG
Processing 2019-08-13-1200/DJI_0084.JPG
Processing 2019-08-13-1200/DJI_0085.JPG
Processing 2019-08-13-1200/DJI_0086.JPG
Processing 2019-08-13-1200/DJI_0087.JPG
Processing 2019-08-13-1200/DJI_0088.JPG
Processing 2019-08-13-1200/DJI_0089.JPG
Processing 2019-08-13-1200/DJI_0090.JPG
Processing 2019-08-13-1200/DJI_0091.JPG
Processing 2019-08-13-1200/DJI_0092.JPG
Processing 2019-08-13-1200/DJI_0093.JPG
Processing 2019-08-13-1200/DJI_0094.JPG
Processing 2019-08-13-1200/DJI_0095.JPG
Processing 2019-08-13-1200/DJI_0096.JPG
Processing 2019-08-13-1200/DJI_0097.JPG
Processing 2019-08-13-1200/DJI_0098.JPG
Processing 2019-08-13-1200/DJI_0099.JPG
Processing 2019-08-13-1200/DJI_0100.JPG
Processing 2019-08-13-1200/DJI_0101.JPG
Processing 2019-08-13-1200/DJI_0102.JPG
Processing 2019-08-13-1200/DJI_0103.JPG
Processing 2019-08-13-1200/DJI_0104.JPG
Processing 2019-08-13-1200/DJI_0105.JPG


...

Processing 2018-12-05-1200/DJI_0293.JPG
Processing 2018-12-05-1200/DJI_0294.JPG
Processing 2018-12-05-1200/DJI_0295.JPG
Processing 2018-12-05-1200/DJI_0296.JPG
Processing 2018-12-05-1200/DJI_0297.JPG
Processing 2018-12-05-1200/DJI_0298.JPG
Processing 2018-12-05-1200/DJI_0299.JPG
Processing 2018-12-05-1200/DJI_0300.JPG
Processing 2018-12-05-1200/DJI_0301.JPG
Processing 2018-12-05-1200/DJI_0302.JPG
Processing 2018-12-05-1200/DJI_0303.JPG
Processing 2018-12-05-1200/DJI_0304.JPG
Processing 2018-12-05-1200/DJI_0305.JPG
Processing 2018-12-05-1200/DJI_0306.JPG
Processing 2018-12-05-1200/DJI_0307.JPG
Processing 2018-12-05-1200/DJI_0308.JPG
Processing 2018-12-05-1200/DJI_0309.JPG
Processing 2018-12-05-1200/DJI_0310.JPG
Processing 2018-12-05-1200/DJI_0311.JPG
Processing 2018-12-05-1200/DJI_0312.JPG
Processing 2018-12-05-1200/DJI_0313.JPG
Processing 2018-12-05-1200/DJI_0314.JPG
Processing 2018-12-05-1200/DJI_0315.JPG
Processing 2018-12-05-1200/DJI_0316.JPG
Processing 2018-12-05-1200/DJI_0317.JPG


Processing 2018-12-05-1200/DJI_0498.JPG
Processing 2018-12-05-1200/DJI_0499.JPG
Processing 2018-12-05-1200/DJI_0500.JPG
Processing 2018-12-05-1200/DJI_0501.JPG
Processing 2018-12-05-1200/DJI_0502.JPG
Processing 2018-12-05-1200/DJI_0503.JPG
Processing 2018-12-05-1200/DJI_0504.JPG
Processing 2018-12-05-1200/DJI_0505.JPG
Processing 2018-12-05-1200/DJI_0506.JPG
Processing 2018-12-05-1200/DJI_0507.JPG
Processing 2018-12-05-1200/DJI_0508.JPG
Processing 2018-12-05-1200/DJI_0509.JPG
Processing 2018-12-05-1200/DJI_0510.JPG
Processing 2018-12-05-1200/DJI_0511.JPG
Processing 2018-12-05-1200/DJI_0512.JPG
Processing 2018-12-05-1200/DJI_0513.JPG
Processing 2018-12-05-1200/DJI_0514.JPG
Processing 2018-12-05-1200/DJI_0515.JPG
Processing 2018-12-05-1200/DJI_0516.JPG
Processing 2018-12-05-1200/DJI_0517.JPG
Processing 2018-12-05-1200/DJI_0518.JPG
Processing 2018-12-05-1200/DJI_0519.JPG
Processing 2018-12-05-1200/DJI_0520.JPG
Processing 2018-12-05-1200/DJI_0521.JPG
Processing 2018-12-05-1200/DJI_0522.JPG


Processing 2019-08-13-1700/DJI_0043.JPG
Processing 2019-08-13-1700/DJI_0044.JPG
Processing 2019-08-13-1700/DJI_0045.JPG
Processing 2019-08-13-1700/DJI_0046.JPG
Processing 2019-08-13-1700/DJI_0047.JPG
Processing 2019-08-13-1700/DJI_0048.JPG
Processing 2019-08-13-1700/DJI_0049.JPG
Processing 2019-08-13-1700/DJI_0050.JPG
Processing 2019-08-13-1700/DJI_0051.JPG
Processing 2019-08-13-1700/DJI_0052.JPG
Processing 2019-08-13-1700/DJI_0053.JPG
Processing 2019-08-13-1700/DJI_0054.JPG
Processing 2019-08-13-1700/DJI_0055.JPG
Processing 2019-08-13-1700/DJI_0056.JPG
Processing 2019-08-13-1700/DJI_0057.JPG
Processing 2019-08-13-1700/DJI_0058.JPG
Processing 2019-08-13-1700/DJI_0059.JPG
Processing 2019-08-13-1700/DJI_0060.JPG
Processing 2019-08-13-1700/DJI_0061.JPG
Processing 2019-08-13-1700/DJI_0062.JPG
Processing 2019-08-13-1700/DJI_0063.JPG
Processing 2019-08-13-1700/DJI_0064.JPG
Processing 2019-08-13-1700/DJI_0065.JPG
Processing 2019-08-13-1700/DJI_0066.JPG
Processing 2019-08-13-1700/DJI_0067.JPG


Processing 2019-08-13-1700/DJI_0248.JPG
Processing 2019-08-13-1700/DJI_0249.JPG
Processing 2019-08-13-1700/DJI_0250.JPG
Processing 2019-08-13-1700/DJI_0251.JPG
Processing 2019-08-13-1700/DJI_0252.JPG
Processing 2019-08-13-1700/DJI_0253.JPG
Processing 2019-08-13-1700/DJI_0254.JPG
Processing 2019-08-13-1700/DJI_0255.JPG
Processing 2019-08-13-1700/DJI_0256.JPG
Processing 2019-08-13-1700/DJI_0257.JPG
Processing 2019-08-13-1700/DJI_0258.JPG
Processing 2019-08-13-1700/DJI_0259.JPG
Processing 2019-08-13-1700/DJI_0260.JPG
Processing 2019-08-13-1700/DJI_0261.JPG
Processing 2019-08-13-1700/DJI_0262.JPG
Processing 2019-08-13-1700/DJI_0263.JPG
Processing 2019-08-13-1700/DJI_0264.JPG
Processing 2019-08-13-1700/DJI_0265.JPG
Processing 2019-08-13-1700/DJI_0266.JPG
Processing 2019-08-13-1700/DJI_0267.JPG
Processing 2019-08-13-1700/DJI_0268.JPG
Processing 2019-08-13-1700/DJI_0269.JPG
Processing 2019-08-13-1700/DJI_0270.JPG
Processing 2019-08-13-1700/DJI_0271.JPG
Processing 2019-08-13-1700/DJI_0272.JPG


Processing 2019-08-13-1700/DJI_0454.JPG
Processing 2019-08-13-1700/DJI_0455.JPG
Processing 2019-08-13-1700/DJI_0456.JPG
Processing 2019-08-13-1700/DJI_0457.JPG
Processing 2019-08-13-1700/DJI_0458.JPG
Processing 2019-08-13-1700/DJI_0459.JPG
Processing 2019-08-13-1700/DJI_0460.JPG
Processing 2019-08-13-1700/DJI_0461.JPG
Processing 2019-08-13-1700/DJI_0462.JPG
Processing 2019-08-13-1700/DJI_0463.JPG
Processing 2019-08-13-1700/DJI_0464.JPG
Processing 2019-08-13-1700/DJI_0465.JPG
Processing 2019-08-13-1700/DJI_0466.JPG
Processing 2019-08-13-1700/DJI_0467.JPG
Processing 2019-08-13-1700/DJI_0468.JPG
Processing 2019-08-13-1700/DJI_0469.JPG
Processing 2019-08-13-1700/DJI_0470.JPG
Processing 2019-08-13-1700/DJI_0471.JPG
Processing 2019-08-13-1700/DJI_0472.JPG
Processing 2019-08-13-1700/DJI_0473.JPG
Processing 2019-08-13-1700/DJI_0474.JPG
Processing 2019-08-13-1700/DJI_0475.JPG
Processing 2019-08-13-1700/DJI_0476.JPG
Processing 2019-08-13-1700/DJI_0477.JPG
Processing 2019-08-13-1700/DJI_0478.JPG


In [ ]:
tagged_image_count = trainer.get_tagged_image_count(custom_vision_para_grass_project_id)
print('Tagged image count:', tagged_image_count)

untagged_image_count = trainer.get_untagged_image_count(custom_vision_para_grass_project_id)
print('Untagged image count:', untagged_image_count)

print('Image count', tagged_image_count + untagged_image_count)